# Прогноз трафика на базовых станциях Мегафон

Импорт библиотек

In [1]:
import numpy as np
import pandas as pd

import os
import tarfile
from six.moves import urllib

from sklearn import tree
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from IPython.display import SVG
from graphviz import Source
from IPython.display import display
from sklearn.model_selection import train_test_split

from sklearn.model_selection import GridSearchCV

from sklearn.metrics import confusion_matrix
import math
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler

import tensorflow as tf
from keras.models import load_model
from keras.models import Sequential
from keras.layers import Dense
import zipfile # Библиотека для работы с zip архивами
from google.colab import drive # Модуль для работы с Google Disk

Чтение тренировочного и тестового датасетов

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
train = pd.read_csv('/content/drive/My Drive/mega/train.gz', index_col=0)
test  = pd.read_csv('/content/drive/My Drive/mega/test_public.gz', index_col=0)

time_idx - номера недель

mb - трафик(целевой признак)

place_id - номер станции

id2 - номер волны(частоты)

col_n00...col_n78 - дополнительные фичи станции


In [ ]:
train.head(2)

,time_idx,mb,place_id,id2,col_n00,col_n01,col_n02,col_n03,col_n04,col_n05,...,col_n69,col_n70,col_n71,col_n72,col_n73,col_n74,col_n75,col_n76,col_n77,col_n78
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
0,8,0.020294,0,5,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.151057,0.0,0.0,0.0,0.0,2.060890
1,9,0.788256,0,5,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.151057,0.0,0.0,0.0,0.0,1.649673


In [ ]:
test.head(2)

,time_idx,place_id,id2,col_n00,col_n01,col_n06,col_n09,col_n10,col_n11,col_n12,...,col_n58,col_n60,col_n61,col_n62,col_n64,col_n69,col_n70,col_n71,col_n73,col_n74
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
34,42,0,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.211308,0.0,0.0,0.0,0.0,0.0,0.21148,0.0
35,43,0,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.211308,0.0,0.0,0.0,0.0,0.0,0.21148,0.0


ПРОПУСКИ

In [ ]:
#Получим долю пропусков в каждом столбце.
cols_null_percent = train.isnull().mean() * 100
cols_with_null = cols_null_percent[cols_null_percent>0].sort_values(ascending=False)
display(cols_with_null)

Series([], dtype: float64)

*Вывод: пропусков нет*

In [ ]:
test['time_idx'].value_counts()

10624    60
608      60
13297    60
5526     60
6037     60
         ..
6553      1
10355     1
2297      1
11188     1
11316     1
Name: place_id, Length: 12523, dtype: int64

*Вывод: Есть станции, которые работали одну неделю*

In [4]:
#1 ФУНКЦИИ

#Функция визуализирует графики, что предсказала сеть и какие были правильные ответы
#start - точка с которой начинаем отрисовку графика
#step - длина графика, которую отрисовываем
#channel - какой канал отрисовываем
def showPredict(start, step, channel, predVal, yValUnscaled):
  plt.plot(predVal[start:start+step, channel],
           label='Прогноз')
  plt.plot(yValUnscaled[start:start+step, channel],
           label='Базовый ряд')
  plt.xlabel('Время')
  plt.ylabel('Значение Close')
  plt.legend()
  plt.show()




#Функция "раскусывания" данных для временных рядов
#data - данные
#xLen - размер "окна", по которому предсказываем
#xChannels - лист, номера каналов, по которым делаем анализ
#yChannels - лист, номера каналов, которые предсказываем
#stepsForward - на сколько шагов предсказываем в будущее
#               если 1 - то на 1 шаг, можно использовать только при одном канале, указанном в yChannels
#xNormalization - нормализация входных каналов, 0 - нормальное распределение, 1 - к отрезку [0;1], -1 - не нормализовывать
#yNormalization - нормализация прогнозируемых каналов, 0 - нормальное распределение, 1 - к отрезку [0;1], -1 - не нормализовывать
#returnFlatten - True - если на выходе получить одномерный вектор для Dense сетей
#valLen - сколько примеров брать для проверочной выборки (количество для обучающей посчитается автоматиески)
#convertToDerivative - bool, преобразовывали ли входные сигналы в производную
def getTrainSeq(data, xLen, xChannels, yChannels, stepsForward, xNormalization, yNormalization, returnFlatten, convertToDerivative):
  #Если указано превращение данных в производную
  #То вычитаем поточечно из текущей точки предыдущую
  if (convertToDerivative):
    data = np.array([(d[1:]-d[:-1]) for d in data.T]).copy().T
  else:
    if isinstance(data,(pd.core.series.Series, pd.core.frame.DataFrame)): #Проверяем, если на входе Pandas - то, берем values, для получения numpy Array
      print("Convert Pandas.Series to Numpy array")
      data = data.values
  #Нормализуем данные
  (xData, xScaler) = DataNormalization(data[-36:-12], xChannels, xNormalization)
  (yData, yScaler) = DataNormalization(data[-12:], yChannels, yNormalization)
  #valLen = 12 #Вычисляем сколько данных с конца нужно взять,
  # чтобы размерность xVal/yVal была valLen
  #Разбивка входного ряда до обработки
  xTrain = xData#[:24]

  yTrain = yData#[24:]

  #xVal = xData[30:]
  #yVal = yData[30:]
  xTrain = np.array([xTrain[i:i + xLen] for i in range(xTrain.shape[0] - xLen + 1 - stepsForward)])
  yTrain = np.array([yTrain[i:i + stepsForward] for i in range(yTrain.shape[0] + 1 - stepsForward)])
  #xVal = np.array([xVal[i:i + xLen, xChannels] for i in range(xVal.shape[0] - xLen + 1 - stepsForward)])
  #yVal = np.array([yVal[i:i + stepsForward] for i in range(xLen, yVal.shape[0] + 1 - stepsForward)])

  #Если в функцию передали вернуть flatten сигнал (для Dense сети), то xTrain и xVal превращаем в flatten
  if (returnFlatten == True):
    xTrain = np.array([x.flatten() for x in xTrain])
    #xVal = np.array([x.flatten() for x in xVal])
  return (xTrain, yTrain), (xScaler, yScaler)#(xVal, yVal),


#Одномерная свертка
def svert1d(xTrain):
  from keras.layers import Dense, Dropout
  from keras.layers import Embedding, Flatten
  from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D

  model = Sequential()
  model.add(Conv1D(100, 3, input_shape = (xLen,
                                        xTrain.shape[2]), activation="linear"))
  model.add(Flatten())
  model.add(Dense(100, activation="LeakyReLU"))
  model.add(Dropout(0.5))
  model.add(Dense(1, activation="linear"))
  model.compile(loss="mse", optimizer=tf.keras.optimizers.Adam(lr=1e-4))

  return (model)


#data - Numpy array
#Normalization = 0 - нормальное распределение, 1 - MinMaxScaler, 2 - RobustScaler, -1 - не нормализовывать
def DataNormalization(datan, Channels, Normalization):
  #Выбираем тип нормализации x
  #0 - нормальное распределение
  #1 - нормирование минимакс
  #2 - robustscaler
  if (Normalization == 0):
    scaler = StandardScaler()
  if (Normalization == 1):
    scaler = MinMaxScaler()
  if (Normalization == 2):
    scaler = RobustScaler()
    #Берём только те каналы, которые указаны в аргументе функции
  resData = datan[:, Channels]
  if (len(resData.shape) == 1): #Если размерность входного массива - одномерный вектор,
    print("Add one dimension")
    resData = np.expand_dims(resData, axis=1) #то добавляем размерность

  if (Normalization == -1):
    scaler = np.zeros(resData.shape[0])
    return (resData, scaler)

  #Обучаем нормировщик
  scaler.fit(resData)
  #Нормируем данные
  resData = scaler.transform(resData)
  return (resData, scaler)

#функция теста
def func_test(data_test, xChannels):

  if (convertToDerivative):
    data_test = np.array([(d[1:] - d[:-1]) for d in data_test.T]).copy().T
  else:
    if isinstance(data_test,(pd.core.series.Series, pd.core.frame.DataFrame)): #Проверяем, если на входе Pandas - то, берем values, для получения numpy Array

      xTest1 = data_test.values
  #Нормализуем данные
  (xTest, xScaler) = DataNormalization(xTest1, xChannels, xNormalization)

  x_Test = np.array([xTest[i:i + xLen] for i in range(xTest.shape[0] - xLen + 2 - stepsForward)])
  return (x_Test, xScaler)

In [5]:
#2 функция обработки данных для traina
# берем значения 36 недель полностью заполненых значениями mb, и которые за это время увеличивалось
# берем максимальные значения признаков и создаем корреляционую матрицу, из которой выбираем
# значения по признаку mb >0.3  <0.9
# далее разбиваем датафрейм для временного ряда, используя 12 недель предсказываем значения на 1 день
def change_data(trainData):
  #убираем столбцы в которых меньше 3 разных значений
  for col in test.columns[3:]:
    if (len(test[col][test['id2']==id].unique())<3):
      test.drop(col, axis=1, inplace=True)

  #убираем столбцы которых нет в тесте  и перемещаем mb и time_idx на 3 и 2 позиции
  tstcol = test.columns
  tstcol = tstcol.drop('time_idx').insert(2, 'time_idx')
  trainData = trainData[tstcol.insert(3, 'mb')]

  #убираем станции с нулевой и отрицательной зависимости(на графике) и которые меньше 36 значений в столбце mb
  for idplace in trainData['place_id'].unique():
    if trainData.loc[(trainData['place_id']==idplace)].shape[0]<36:
      trainData.drop(trainData[(trainData['place_id']==idplace)].index, axis=0, inplace=True)  #&(trainData['id2']==id)

    if (trainData['mb'][(trainData['place_id']==idplace)&(trainData['time_idx']==6)].values >= trainData['mb'][(trainData['place_id']==idplace)&(trainData['time_idx']==41)].values):
      trainData.drop(trainData[trainData['place_id']==idplace].index, axis=0, inplace=True)

    if (trainData['mb'].loc[(trainData['place_id']==idplace)].all()==0):
      trainData.drop(trainData[trainData['place_id']==idplace].index, axis=0, inplace=True)

  #убираем столбцы в которых меньше 3 разных значений
  for col in trainData.columns[3:]:
    if (len(trainData[col].unique())<3):
      trainData.drop(col, axis=1, inplace=True)

  #вычисляем максимальные значения по столбцам, сгруппированным по неделям
  x_trainData = pd.DataFrame(index = trainData['time_idx'].unique(), columns = trainData.iloc[:, 2:trainData.shape[1]].columns)
  x_trainData = trainData.iloc[:, 2:trainData.shape[1]].groupby(['time_idx']).max()

  # МАТРИЦА КОРРЕЛЯЦИИ
  correlation_matrix = x_trainData.corr()

  #выборка из корреляционной матрицы параметры значения корреляции которых по mb>0.3 и <0.9
  corr_df = correlation_matrix['mb'].loc[(correlation_matrix['mb']>0.3)?(correlation_matrix['mb']<0.9)]  #.sort_values(ascending=False)

  #формирование датафрейма по этим параметрам
  x_trainData = pd.DataFrame(trainData, columns = corr_df.index)

  xChannels = range(1, x_trainData.shape[1]) #Используемые входные каналы

  #разбивка на трайн и (валидационную выборку можно сделать когда будет 48 месяцев трайна)
  (xTrain, yTrain), (xScaler, yScaler) = getTrainSeq(x_trainData, xLen, xChannels, yChannels, stepsForward,
                                                     xNormalization, yNormalization, returnFlatten, convertToDerivative)

  return (xTrain, yTrain), (xScaler, yScaler), x_trainData.columns[1:]


In [6]:
#3
#Параметры обучения модели
xLen = 12 #Анализируем по 12 прошедшим точкам
stepsForward = 1            #Тренируем сеть для предсказания на 1 шаг
#xChannels =range(1,x_train36id2_6.shape[1]) #Используемые входные каналы
#xtestChannels =range(3,40) #Используем все входные каналы
yChannels = range(0, 1) #Предсказываем только канал
xNormalization = 2          #Нормируем входные каналы стандартным распределением
yNormalization = 2          #Нормируем выходные каналы стандартным распределением
#valLen = 3              #Используем 3 записей для проверки
returnFlatten = False       #Если True, то вернуть одномерные векторы, если False, то двумерные
convertToDerivative = False #Не True, то превращать в производную

#Компилируем модель обучения
dict_id2 = dict.fromkeys(train['id2'].unique())

for id in train['id2'].unique():

  (xTrain, yTrain), (xScaler, yScaler), id2_columns = change_data(train[(train['id2']==id)])
  dict_id2[id] = id2_columns

  modelC = svert1d(xTrain)

  #обучение модели
  history = modelC.fit(xTrain, yTrain, epochs = 100, batch_size = 1, verbose = 1)
  modelC.save("/content/drive/My Drive/mega/modelC" + str(id) + ".h5")


<ipython-input-5-75a3d71b21c1>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trainData.drop(trainData[(trainData['place_id']==idplace)].index, axis=0, inplace=True)  #&(trainData['id2']==id)
<ipython-input-5-75a3d71b21c1>:18: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if (trainData['mb'][(trainData['place_id']==idplace)&(trainData['time_idx']==6)].values >= trainData['mb'][(trainData['place_id']==idplace)&(trainData['time_idx']==41)].values):
<ipython-input-5-75a3d71b21c1>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_g

Convert Pandas.Series to Numpy array


Epoch 1/100
12/12 [==============================] - 2s 12ms/step - loss: 1.5669
Epoch 2/100
12/12 [==============================] - 0s 9ms/step - loss: 1.0630
Epoch 3/100
12/12 [==============================] - 0s 9ms/step - loss: 0.7097
Epoch 4/100
12/12 [==============================] - 0s 9ms/step - loss: 1.2546
Epoch 5/100
12/12 [==============================] - 0s 10ms/step - loss: 0.2857
Epoch 6/100
12/12 [==============================] - 0s 8ms/step - loss: 0.2526
Epoch 7/100
12/12 [==============================] - 0s 9ms/step - loss: 0.3055
Epoch 8/100
12/12 [==============================] - 0s 9ms/step - loss: 0.0879
Epoch 9/100
12/12 [==============================] - 0s 9ms/step - loss: 0.0952
Epoch 10/100
12/12 [==============================] - 0s 8ms/step - loss: 0.3842
Epoch 11/100
12/12 [==============================] - 0s 8ms/step - loss: 0.2449
Epoch 12/100
12/12 [==============================] - 0s 9ms/step - loss: 0.1151
Epoch 13/100
12/12 [===============

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
<ipython-input-5-75a3d71b21c1>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trainData.drop(trainData[trainData['place_id']==idplace].index, axis=0, inplace=True)
<ipython-input-5-75a3d71b21c1>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trainData.drop(trainData[trainData['place_id']==idplace].index, axis=0, inp

Convert Pandas.Series to Numpy array
Epoch 1/100
12/12 [==============================] - 1s 6ms/step - loss: 1.6791
Epoch 2/100
12/12 [==============================] - 0s 4ms/step - loss: 8.5339
Epoch 3/100
12/12 [==============================] - 0s 5ms/step - loss: 2.2077
Epoch 4/100
12/12 [==============================] - 0s 5ms/step - loss: 1.9512
Epoch 5/100
12/12 [==============================] - 0s 4ms/step - loss: 19.1305
Epoch 6/100
12/12 [==============================] - 0s 4ms/step - loss: 8.4532
Epoch 7/100
12/12 [==============================] - 0s 5ms/step - loss: 11.8507
Epoch 8/100
12/12 [==============================] - 0s 4ms/step - loss: 2.6629
Epoch 9/100
12/12 [==============================] - 0s 7ms/step - loss: 2.3827
Epoch 10/100
12/12 [==============================] - 0s 5ms/step - loss: 1.0820
Epoch 11/100
12/12 [==============================] - 0s 5ms/step - loss: 1.3976
Epoch 12/100
12/12 [==============================] - 0s 5ms/step - loss: 2.574

...

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
<ipython-input-5-75a3d71b21c1>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trainData.drop(trainData[trainData['place_id']==idplace].index, axis=0, inplace=True)
<ipython-input-5-75a3d71b21c1>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trainData.drop(trainData[trainData['place_id']==idplace].index, axis=0, inp

Convert Pandas.Series to Numpy array
Epoch 1/100
12/12 [==============================] - 1s 5ms/step - loss: 6.3013
Epoch 2/100
12/12 [==============================] - 0s 5ms/step - loss: 9.9169
Epoch 3/100
12/12 [==============================] - 0s 5ms/step - loss: 2.0106
Epoch 4/100
12/12 [==============================] - 0s 5ms/step - loss: 6.0685
Epoch 5/100
12/12 [==============================] - 0s 5ms/step - loss: 7.3699
Epoch 6/100
12/12 [==============================] - 0s 6ms/step - loss: 13.0207
Epoch 7/100
12/12 [==============================] - 0s 5ms/step - loss: 4.0311
Epoch 8/100
12/12 [==============================] - 0s 6ms/step - loss: 8.6750
Epoch 9/100
12/12 [==============================] - 0s 6ms/step - loss: 2.7544
Epoch 10/100
12/12 [==============================] - 0s 5ms/step - loss: 6.2898
Epoch 11/100
12/12 [==============================] - 0s 5ms/step - loss: 9.3345
Epoch 12/100
12/12 [==============================] - 0s 5ms/step - loss: 2.3933

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
<ipython-input-5-75a3d71b21c1>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trainData.drop(trainData[trainData['place_id']==idplace].index, axis=0, inplace=True)
<ipython-input-5-75a3d71b21c1>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trainData.drop(trainData[trainData['place_id']==idplace].index, axis=0, inp

Convert Pandas.Series to Numpy array
Epoch 1/100
12/12 [==============================] - 1s 4ms/step - loss: 5.1974
Epoch 2/100
12/12 [==============================] - 0s 6ms/step - loss: 4.8132
Epoch 3/100
12/12 [==============================] - 0s 6ms/step - loss: 2.2232
Epoch 4/100
12/12 [==============================] - 0s 6ms/step - loss: 2.4366
Epoch 5/100
12/12 [==============================] - 0s 6ms/step - loss: 2.3522
Epoch 6/100
12/12 [==============================] - 0s 5ms/step - loss: 3.7381
Epoch 7/100
12/12 [==============================] - 0s 5ms/step - loss: 4.0549
Epoch 8/100
12/12 [==============================] - 0s 6ms/step - loss: 2.9766
Epoch 9/100
12/12 [==============================] - 0s 7ms/step - loss: 1.2677
Epoch 10/100
12/12 [==============================] - 0s 6ms/step - loss: 1.2469
Epoch 11/100
12/12 [==============================] - 0s 6ms/step - loss: 2.3493
Epoch 12/100
12/12 [==============================] - 0s 6ms/step - loss: 0.6344


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [7]:
# функция предсказания по 12 неделям для одной волны по одной станции
def func_pred(tmp_df, idplace):
  #выделяем выборку из теста под трайн и предсказываем
  idplace_tmp_df = pd.DataFrame(tmp_df.loc[tmp_df['place_id']==idplace], columns = dict_id2[id])
  idplace_tmp_df.fillna(0, inplace=True)
  #убираем станции если меньше 12 недель
  if idplace_tmp_df.shape[0]==12:
    (xTmp, tmpScaler_test) = func_test(idplace_tmp_df[-12:], range(idplace_tmp_df.shape[1]))
    pred_idplace = yScaler.inverse_transform(modelC.predict(xTmp))
    return pred_idplace


pred_df = test[['place_id', 'id2']].copy()
pred_df['predval'] = 0
for id in test['id2'].unique():
  #выделяем выборку из теста
 # test_tmp1 = pd.DataFrame(test.loc[test['id2']==id])
  test_tmp1 = test.loc[test['id2']==id]
  #test_tmp1.fillna(0, inplace=True)

  modelC = load_model('/content/drive/My Drive/mega/modelC'+str(id)+'.h5') # загружаем модель для волны

  for idplace in test_tmp1['place_id'].unique():

    predVal = func_pred(test_tmp1, idplace)
    pred_df['predval'].loc[(pred_df['id2']==id)&(pred_df['place_id']==idplace)] = predVal[0][0]

Выходные данные были обрезаны до нескольких последних строк (5000).
1/1 [==============================] - 0s 29ms/step


Сохранить предсказание

In [10]:
pred_df.to_csv('/content/drive/My Drive/mega/submit.csv')